In [206]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import praw
import datetime as dt
import requests
import numpy as np
import pytz
import time
import yfinance as yf




# Convert from date to unix and unix to date

In [42]:
def convert_to_unix(date):
  unixd = dt.datetime.strptime(date,"%Y-%m-%d")
  return time.mktime(unixd.timetuple())

  
def convert_to_date(timestamp):
  return str(dt.datetime.fromtimestamp(int(timestamp)).date())

In [6]:
convert_to_unix('2020-02-20')

1582128000.0

# Use pushshift api to retrieve reddit data

In [203]:
query = "bitcoin"
subreddit="bitcoin" #Define Your subreddit
after="2d"
before = "1d"
fields = "title,created_utc"
sort_type = "created_utc" #score, num_comments, default=created_utc
def get_data(query,
            subreddit,
            before,
            after,
            fields,
            size,
            sort_type):
    url = f"https://api.pushshift.io/reddit/search/submission/?q={query}&before={before}&after={after}&subreddit={subreddit}&fields={fields}&size={size}&sort_type={sort_type}"
    request = requests.get(url)
    json_response = request.json()
    return json_response['data']

def multiple_data_call(lookback_period:int,
                **kwargs):
        df = pd.DataFrame(columns = ['date','reddit_title'])
        for i in range(1,lookback_period+1):
                after = str(i+1)+'d'
                before = str(i)+'d'
                response=get_data(query,subreddit,before,after,fields,size,sort_type)
                titles = []
                date = []         
                for line in response:
                        date.append(convert_to_date(line['created_utc']))
                        titles.append(line['title'])
                temp = pd.DataFrame(np.column_stack([date,titles]),columns = ['date','reddit_title'])
                df = pd.concat([df,temp])
        df.index = list(range(len(df)))
        return df
                


In [224]:
tickerlist = ['BTC-USD']
df = yf.download(tickerlist, period = '1d', start = '2007-01-01',end = pd.Timestamp.today())
columns = df.columns
df = df.iloc[:,2]['2020':]
df

[*********************100%***********************]  1 of 1 completed


Date
2020-01-01     7174.944336
2020-01-02     6935.270020
2020-01-03     6914.996094
2020-01-04     7309.514160
2020-01-05     7400.535645
                  ...     
2022-03-17    40662.871094
2022-03-18    40302.398438
2022-03-19    41602.667969
2022-03-20    41004.757812
2022-03-21    40691.011719
Name: Low, Length: 811, dtype: float64

In [248]:
pct_change = pd.DataFrame(df.pct_change().fillna(method = 'bfill'))
pct_change.columns = ['pct_change']
pct_change['date'] = pct_change.index.astype(str)
pct_change['index'] = range(len(pct_change))
pct_change.set_index('index',drop = True,inplace = True)
pct_change

,pct_change,date
index,,
0,-0.033404,2020-01-01
1,-0.033404,2020-01-02
2,-0.002923,2020-01-03
3,0.057053,2020-01-04
4,0.012452,2020-01-05
...,...,...
806,0.042041,2022-03-17
807,-0.008865,2022-03-18
808,0.032263,2022-03-19


In [253]:
query = "bitcoin"
subreddit="bitcoin" #Define Your Query
fields = "title,created_utc"
size = 1
sort_type = "score" #score, num_comments, default=created_utc
d10_data = multiple_data_call(lookback_period=20,
            query = query,
            subreddit=subreddit,
            fields=fields,
            sort_type=sort_type)
d10_data

,date,reddit_title
0,2022-03-19,"March 19, 2013: Bitcoin and me (Hal Finney)"
1,2022-03-18,My Biggest Regret (story and advice from a sil...
2,2022-03-18,"You already missed the bus with bitcoin, don’t..."
3,2022-03-17,Is Bitcoin still a good investment in 2022?
4,2022-03-16,BITCOIN BITCOIN BITCOIN!!!
5,2022-03-15,What do you think would happen to the value of...
6,2022-03-13,Bitcoin assassination markets could quickly en...
7,2022-03-12,Best YouTube channels for regular Bitcoin TA
8,2022-03-12,Reminder that 50% of all purchases in the US a...
9,2022-03-11,What can we do to stop the powerful people fro...


In [254]:
d10_data = d10_data.merge(pct_change, on='date',how = 'left')
d10_data


,date,reddit_title,pct_change
0,2022-03-19,"March 19, 2013: Bitcoin and me (Hal Finney)",0.032263
1,2022-03-18,My Biggest Regret (story and advice from a sil...,-0.008865
2,2022-03-18,"You already missed the bus with bitcoin, don’t...",-0.008865
3,2022-03-17,Is Bitcoin still a good investment in 2022?,0.042041
4,2022-03-16,BITCOIN BITCOIN BITCOIN!!!,0.018589
5,2022-03-15,What do you think would happen to the value of...,0.016706
6,2022-03-13,Bitcoin assassination markets could quickly en...,-0.026936
7,2022-03-12,Best YouTube channels for regular Bitcoin TA,0.011086
8,2022-03-12,Reminder that 50% of all purchases in the US a...,0.011086
9,2022-03-11,What can we do to stop the powerful people fro...,-0.012502


# Workings

In [ ]:
titles = []
date = []
for data in json_response['data']:
    date.append(convert_to_date(data['created_utc']))
    titles.append(data['title'])
df = pd.DataFrame(titles, index = date,columns = ['reddit_title'])
df

In [218]:
list1 = [1,2,3,4]
list2= map(lambda x:x*2,list1)
list(list2)

[2, 4, 6, 8]